This is a program developed to solve acrostics (with input, not automatically).

In [1]:
class Acrostic:
    def __init__(self, clues, word_lengths, sol_enum):
        self.clues = clues
        self.solution_enum = sol_enum
        self.num_to_clue = {}
        #takes number to clue, position in clue
        for i in range(len(clues)):
            for j in range(len(clues[i])):
                self.num_to_clue[clues[i][j]] = [i, j]
        self.clue_dict = {tuple(v): k for k, v in self.num_to_clue.items()}
        #takes clue, position in clue to number
        self.num_to_wd = {}
        #takes number to word in solution, position in that word
        count = 1
        for i in range(len(word_lengths)):
            for j in range(word_lengths[i]):
                self.num_to_wd[count] = [i, j]
                count += 1
        self.wd_dict = {tuple(v):k for k, v in self.num_to_wd.items()}
        #takes word in solution and its position to number
        self.solution = []
        for k in word_lengths:
            self.solution.append("."*k)
        self.clue_list = []
        for clue in clues:
            self.clue_list.append("."*len(clue))
                
    def print_sol(self):
        return " ".join(self.solution)
    
    def print_clues(self):
        return "\n".join(self.clue_list)
    
    def print_info(self):
        return [self.num_to_clue, self.clue_dict, self.num_to_wd, self.wd_dict]

The class Acrostic contains several variables and functions. The main features are dictionaries that connect the numbers in an acrostic to both their location in the solution (straightforward) and their location in the clue answers (slightly harder). It also has functions to print the current status of the quote and of the clues, plus a diagnostic function that I used as a check. The majority of the information contained here is the initial setup, with the majority of the work done by the next few functions.

Note that the actual clues are not included here, just the enumerations. Rather than storing the clues, it is intended to refer to an already written list of clues.

In [2]:
def update_clue(acrostic, clue_num, answer): #changes answer to a clue
    acrostic.clue_list[clue_num] = answer
    return acrostic

def update_solution(acrostic, word_num, answer): #changes word in solution
    acrostic.solution[word_num] = answer
    return acrostic

def update_db_clue(acrostic): #updates solution when clue is changed
    for clue in range(len(acrostic.clue_list)):
        for letter in range(len(acrostic.clue_list[clue])):
            a,b = acrostic.num_to_wd[acrostic.clue_dict[(clue, letter)]]
            acrostic.solution[a] = acrostic.solution[a][:b] + acrostic.clue_list[clue][letter] + acrostic.solution[a][b+1:]
    return acrostic

def update_db_sol(acrostic): #updates clues when solution word is changed
    for word in range(len(acrostic.solution)):
        for letter in range(len(acrostic.solution[word])):
            a,b = acrostic.num_to_clue[acrostic.wd_dict[(word, letter)]]
            acrostic.clue_list[a] = acrostic.clue_list[a][:b] + acrostic.solution[word][letter] + acrostic.clue_list[a][b+1:]
    return acrostic

The first two functions are used to update the clue list and the quote, respectively. The latter two are used to update the quote when a new answer is added or vice versa. The aforementioned dictionaries are used to link the two.

The next functions are used with a file as input. The text file has the enumeration of the quote as its first line, and each subsequent line is the numbers used (in order) for a given clue.

In [3]:
def acrostic_file(file): 
    #file is text file, 1st line is enumeration of quote, 
    #other lines are numbers associated with each clue
    f = open(file, "r")
    lines = []
    for line in f:
        lines.append(line.strip())
    f.close()
    wordlengths = lines[0]
    word_lengths = [int(length) for length in wordlengths.split()]
    sol_enum = [sum(word_lengths[:i+1]) for i in range(len(word_lengths))]
    clues = []
    for i in range(1, len(lines)):
        clue = lines[i]
        clues.append([int(j) for j in clue.split()])
    
    
    acrostic = Acrostic(clues, word_lengths, sol_enum)
    print(acrostic.print_sol())
    print(acrostic.print_clues())
    return acrostic

def from_file(file):
    acrostic = acrostic_file(file)
    print("1: Change a word in the acrostic")
    print("2: Change an answer to a clue")
    print("3: Exit program")
    while True:
        option = input("Which option? ")
        if option == "3":
            break
        elif option == "1":
            word_num = input("Which word? ")
            answer = input("New answer: ")
            acrostic = update_solution(acrostic, int(word_num)-1, answer)
            acrostic = update_db_sol(acrostic)
        elif option == "2":
            clue_num = input("Which clue? ")
            answer = input("New answer: ")
            acrostic = update_clue(acrostic, int(clue_num)-1, answer)
            acrostic = update_db_clue(acrostic)
        else:
            print("Invalid choice")
        print(acrostic.print_sol())
        print(acrostic.print_clues())

The next two functions were the originals I created, and involved entering the numbers associated with each clue manually. This was a pain to fix if a mistake was made, so I refitted it to work with a text file input, which is easier to fix. The overall structure is the same.

In [4]:
def init(numclues, wordlengths):
    word_lengths = [int(length) for length in wordlengths.split()]
    sol_enum = [sum(word_lengths[:i+1]) for i in range(len(word_lengths))]
    clues = []
    for i in range(numclues):
        clue = input("Which numbers for clue {}? ".format(i+1))
        clues.append([int(j) for j in clue.split()])
    
    
    acrostic = Acrostic(clues, word_lengths, sol_enum)
    print(acrostic.print_sol())
    print(acrostic.print_clues())
    return acrostic

def overall_with_input(numclues, wordlengths):
    acrostic = init(numclues, wordlengths)
    print("1: Change a word in the acrostic")
    print("2: Change an answer to a clue")
    print("3: Exit program")
    while True:
        option = input("Which option? ")
        if option == "3":
            break
        elif option == "1":
            word_num = input("Which word? ")
            answer = input("New answer: ")
            acrostic = update_solution(acrostic, int(word_num)-1, answer)
            acrostic = update_db_sol(acrostic)
        elif option == "2":
            clue_num = input("Which clue? ")
            answer = input("New answer: ")
            acrostic = update_clue(acrostic, int(clue_num)-1, answer)
            acrostic = update_db_clue(acrostic)
        else:
            print("Invalid choice")
        print(acrostic.print_sol())
        print("") #line break
        print(acrostic.print_clues())

One future feature to work on is protecting the clue list/quote so that it is impossible to enter a word of incorrect length. A better interface would be nice but is out of my area of expertise.

In [5]:
from_file("acrostic.txt")

...... ... ....... ... .. ... ..... .... .. .. ......... ....... ........ .. ..... .... ... ..... ... .. ..... ..... .. .... ... ....... .. .... .... ... ........ .... .. .. ......... .... .... .. ... ... ....
.....
..........
.......
....
........
.......
.......
....
.......
.........
.......
.......
.......
....
......
.....
.........
....
.....
......
......
.......
......
.........
.....
........
1: Change a word in the acrostic
2: Change an answer to a clue
3: Exit program
Which option? 2
Which clue? 2
New answer: EDWINBOOTH
...O.. ... ....... ... I. ... ..... .... T. .. ......... ......N ........ .. ....D .... ... ..... H.. .. ..... ..... .. ..O. ... ....... .. .... .... W.. ........ .... .. .. ......... .E.. .... .. ... ... B...
.....
EDWINBOOTH
.......
....
........
.......
.......
....
.......
.........
.......
.......
.......
....
......
.....
.........
....
.....
......
......
.......
......
.........
.....
........
Which option? 2
Which clue? -1
New answer: BANG
...O.. ...